In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Basis State preparation for the two numbers
def state_preparation(basis_id, n_qubits):
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    return qml.BasisStatePreparation(bits, wires=range(n_qubits))
state_preparation(3,5)

BasisStatePreparation([0, 0, 0, 1, 1], wires=[0, 1, 2, 3, 4])

In [3]:
# Binary to decimal
def binary_to_string(a):
#     for i in a[0]:
    listToStr = ' '.join([str(elem) for elem in a[0]])
    print(len(listToStr))
    return int(listToStr,2)
# binary_to_string()

In [4]:
# Swappping of quantum circuits 
def swap_bits(n_qubits):
    """A circuit that reverses the order of qubits, i.e.,
    performs a SWAP such that [q1, q2, ..., qn] -> [qn, ... q2, q1].
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    for i in range(int(n_qubits/2)):
        qml.SWAP(wires=[i,n_qubits-1-i])

In [5]:
# Conditional Rotation matrix
def qft_rotations(n_qubits):
    """A circuit performs the QFT rotations on the specified qubits.
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    n = n_qubits
    for i in range(n):
        qml.Hadamard(wires=i)
        for jj in range(i+1,n,1):
            qml.ControlledPhaseShift(np.pi/(2**(jj-i)), wires=[jj,i])
    pass

In [8]:
def qft_node(basis_id, basis_id2, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    
    bits2 = [int(x) for x in np.binary_repr(basis_id2, width=n_qubits)]
    qml.BasisStatePreparation(bits2, wires=range(n_qubits, 2*(n_qubits),1))
    qft_rotations(n_qubits)
    for i in range(0,n_qubits,1):
        k=0
        for j in range(i+n_qubits,(2*n_qubits),1):
            qml.ControlledPhaseShift((np.pi/(2**(k))), wires=[j,i])
            k+=1
    (qml.adjoint)(qft_rotations)(n_qubits)
    return qml.sample(wires=[x for x in range(n_qubits)])

# basis_id stores a, basis_id2 stores b
basis_id = int(input("Enter number a: "))
basis_id2 = int(input("Enter number b: "))
if len(np.binary_repr(basis_id))>len(np.binary_repr(basis_id2)):
    n_qubits = len(np.binary_repr(basis_id)) + 1
else:
    n_qubits = len(np.binary_repr(basis_id2)) + 1
dev = qml.device('default.qubit', wires=(2*n_qubits), shots=10)
qnode = qml.QNode(qft_node, dev)

k = qnode(basis_id, basis_id2, n_qubits)
poww, summ = 0, 0
for l in reversed(k[0]):
    summ = summ + (2**poww)*l
    poww+=1
print(summ)

Enter number a: 3
Enter number b: 2
5


In [ ]:
def multiplier(basis_id, basis_id2, n_qubits):
    bits1 = [int(x) for x in np.binary_repr(0, width=2*n_qubits)]
    qml.BasisStatePreparation(bits1, wires=range(2*(n_qubits)))
    
    bits2 = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits2, wires=range(2*n_qubits, 3*n_qubits))
    
    bits3 = [int(x) for x in np.binary_repr(basis_id2, width=n_qubits)]
    qml.BasisStatePreparation(bits3, wires=range(3*n_qubits, 4*(n_qubits),1))
       
    qft_rotations(2*n_qubits)
    
    for i in range(3*n_qubits, 4*n_qubits,1):
        for j in range(2*n_qubits, 3*n_qubits,1):
            for m in range(2*n_qubits):
                qml.ctrl(qml.PhaseShift(np.pi/(2**(-(5*n_qubits)-m+j+i+1)), wires=m),control=[j,i])
    (qml.adjoint)(qft_rotations)(2*n_qubits)
    return qml.sample(wires=[x for x in range(2*n_qubits)])

basis_id = int(input("Enter number a: "))
basis_id2 = int(input("Enter number b: "))
if len(np.binary_repr(basis_id))>len(np.binary_repr(basis_id2)):
    n_qubits = len(np.binary_repr(basis_id)) 
else:
    n_qubits = len(np.binary_repr(basis_id2))
dev = qml.device('default.qubit', wires=(4*n_qubits), shots=10)
qnode = qml.QNode(multiplier, dev)

# Below code returns the sample containing measurements on the wires
# print(qnode(basis_id,basis_id2,n_qubits))  
# Below prints quantum circuit
# print(qml.draw(qnode)(basis_id,basis_id2,n_qubits))

k = qnode(basis_id, basis_id2, n_qubits)
poww, summ = 0, 0
for l in reversed(k[0]):
    summ = summ + (2**poww)*l
    poww+=1
print(summ)